# Goal: predict whether a loan will end up with maximum profits or not

---
#### Target variable: `zeroBalCode` 
* Type: **Categorical** 
* Model type: Classification 
* Sourced from: `zeroBalCode`
* Data: 
    - "0" means "Successful outcome for Fannie Mae"
    - "1" means "Negative outcome for Fannie Mae"
    
#### This Notebook:
* Input required: The output .pkl model file from "Scott - Model - 1- PyCaret Setup and Create Model" notebook
* Outputs generated: None

#### Expected Workflow
1. Scott - Data Pre - 1 - Feature EEE
2. Scott - Data Pre - 2 - 50 50 split train test
3. Scott - Model - 1- PyCaret Setup and Create Model
4. Scott - Predict - 1 - Holdout Data

In [1]:
# Dependencies
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_auc_score
import numpy as np

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter("ignore", category=DeprecationWarning) 
warnings.simplefilter("ignore", UserWarning)

import winsound

# Tell Jupyter to display all text, not just "the last" and print()
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

%pwd

def DoneNotice(duration_ms = 1000):
    duration = duration_ms  # milliseconds
    freq = 440  #Hz
    winsound.Beep(freq, duration)

from IPython.display import Markdown, display
def Important(html_tag, message, color):
    colorstr = f"<{html_tag} style='color:{color}'>{message}</{html_tag}>"
    display(Markdown(colorstr))

# Load the model

In [2]:
# .pkl will be automatically added by PyCaret
model = '../models/et20200526_2010'

# What is the model's target variable?
target = 'zeroBalCode'

# What are the inputs to the model?
predictors = [
    'origChannel'
    , 'loanPurp'
    , 'bankNumber'
    , 'stateNumber'
    , 'mSA'
    , 'origIntRate'
    , 'origUPB'
    , 'origLTV'
    , 'numBorrowers'
    , 'origDebtIncRatio'
    , 'worstCreditScore'
]

# Where is your holdout data?
holdoutData = '../data/DataPre-01-Feature-EEE-2012.csv'

In [3]:
%%time 

from pycaret.classification import *
the_model = load_model(model)

DoneNotice(2000)

print(the_model)

Transformation Pipeline and Model Sucessfully Loaded


[Pipeline(memory=None,
         steps=[('dtypes',
                 DataTypes_Auto_infer(categorical_features=['origChannel',
                                                            'loanPurp',
                                                            'bankNumber',
                                                            'stateNumber',
                                                            'mSA'],
                                      display_types=False, features_todrop=[],
                                      ml_usecase='classification',
                                      numerical_features=['origIntRate',
                                                          'origUPB', 'origLTV',
                                                          'numBorrowers',
                                                          'origDebtIncRatio',
                                                          'worstCreditScore'],
                                      target='zeroBalCod

# Load in holdout data (2011 data)

In [4]:
%%time

dfHoldout = pd.read_csv(holdoutData)
dfHoldout.head()

Wall time: 32.9 ms


,Unnamed: 0,origChannel,origIntRate,origUPB,origLTV,numBorrowers,origDebtIncRatio,loanPurp,worstCreditScore,bankNumber,stateNumber,mSA,zeroBalCode
0,92143,3,5.250,46000,80,2,25,1,691,80,25,44180,0
1,92144,1,4.375,219000,70,2,44,1,752,80,32,35620,1
2,92145,3,4.875,76000,78,1,38,1,692,29,44,16860,1
3,92147,1,4.250,101000,75,1,41,1,686,54,36,0,0
4,92148,1,4.375,256000,80,1,37,1,723,45,19,35380,0


In [5]:
# Drop the previous index column
dfHoldout.drop(['Unnamed: 0'], axis=1, inplace=True)

# Predict!
Notice the last two columns 'Label' and 'Score'. 
- Label is the prediction 
- Score is the probability of the prediction
The predicted results are concatenated to the original dataset while all transformations including imputation of missing values (in this case None), categorical encoding, feature extraction etc. are performed automatically under the hood and you do not have to manage the pipeline manually.

In [6]:
%%time 

unseen_predictions = predict_model(model, data=dfHoldout)
unseen_predictions.head()

DoneNotice(2000)

Wall time: 3.6 s


In [7]:
results = unseen_predictions[[target,'Label','Score']]

In [8]:
def calc_confusion(row):
    if ((row[target] == 0) & (row['Label'] == 0)):
        value = 'TrueNegative'
    elif ((row[target] == 0) & (row['Label'] == 1)):
        value = 'FalseNegative'
    elif ((row[target] == 1) & (row['Label'] == 1)):
        value = 'TruePositive'
    elif ((row[target] == 1) & (row['Label'] == 0)):
        value = 'FalsePositive'
    else:
        value = 'Undefined'
    return value

results['Confusion'] = results.apply(calc_confusion, axis=1)

confusionMatrix = results.Confusion.value_counts().to_dict()

In [9]:
def makeCell(input, position):
    actual = len(str(input))
    if actual == 7: # 3 and 3
        return_string = (" " * 3) + str(input) + (" " * 3)
    elif actual == 6: # 3 and 4
        return_string = (" " * 3) + str(input) + (" " * 4)
    elif actual == 5: # 4 and 4
        return_string = (" " * 4) + str(input) + (" " * 4)
    elif actual == 4: # 4 and 5
        if position == 'left':
            return_string = (" " * 4) + str(input) + (" " * 5)
        else:
            return_string = (" " * 4) + str(input) + (" " * 6)
    elif actual == 3: # 5 and 5
        if position == 'left':
            return_string = (" " * 5) + str(input) + (" " * 5)
        else:
            return_string = (" " * 5) + str(input) + (" " * 6)
    elif actual == 2: # 5 and 6
        if position == 'left':
            return_string = (" " * 5) + str(input) + (" " * 6)
        else:
            return_string = (" " * 5) + str(input) + (" " * 7)
    else: # 1: 6 and 6
        if position == 'left':
            return_string = (" " * 6) + str(input) + (" " * 6)
        else:
            return_string = (" " * 6) + str(input) + (" " * 7)

    return return_string

def make_confusion_matrix(tn, fp, fn, tp):
    print(f'           +----------------------------+')
    print(f'           |(TN)         |          (FP)|')
    print(f'         0 |{makeCell(tn, "left")}|{makeCell(fp, "right")}|')
    print(f'           |             |              |')
    print(f' Actual    |-------------|--------------|')
    print(f'           |             |              |')
    print(f'         1 |{makeCell(fn, "left")}|{makeCell(tp, "right")}|')
    print(f'           |(FN)         |          (TP)|')
    print(f'           |_____________|______________|')
    print(f'                  0              1       ')
    print(f'                     Predicted           ')
          
def getAccuracy(tp, tn, fp, fn, decimals):
    try:
        recall_sensitivity = tp / float(tp + fn)
    except:
        recall_sensitivity = 0

    try:
        precision = tp / float(tp + fp)
    except:
        precision = 0

    try:
        fscore = 2 * (precision * recall_sensitivity / precision + recall_sensitivity)
        # fscore = 2*precision*recall / (precision + recall)
    except:
        fscore = 0

    try:
        accuracy = (tp + tn) / float(tp + tn + fp + fn)
    except:
        accuracy = 0
          
    try:
        specificity = tn / float(tn + fp)
    except:
        specificity = 0
          
    try:
        balanced_accuracy = (recall_sensitivity + specificity) / float(2)
    except:
        balanced_accuracy = 0
          
    # Specificity
    # Specificity is the correctly -ve labeled by the program to all who are healthy in reality.
    # Specifity answers the following question: Of all the people who are healthy, how many of those did we correctly predict?    
          
    # Precision = Ability of the classifier not to label as positive a sample that is negative.
          
    # Recall = Sensitivity = Ability of the classifier to find all the positive samples.
          
    # Balanced Accuracy = (sensitivity + specificity) / 2
          
    # https://stats.stackexchange.com/questions/49579/balanced-accuracy-vs-f-1-score
    # Both F1 and Balanced Accuracy both (to some extent) handle class imbalance. For binary classification, 
    #     depending of which of the two classes (N or P) outnumbers the other, each metric outperforms the other:
    #     - If N > P, F1 is better
    #     - If P > N, Balanced Accuracy is better
    # Clearly, if you can label-switch, both the metrics can be used in any of the two imbalance cases above. 
    # If not, then depending on the imbalance in the training data, you can select the appropriate metric.
    # 
    # Balanced Accuracy = arithmetic mean of Recall for Positive and Negative
    # F1 = harmonic mean of Recall_P and Precision_P
    # 
    # Balanced Accurancy = (Sensitivity + Specificity) / 2
    # F1 = 

    # Sensitivity is Recall
          
    # Specificity is the false positive rate. Sensitivity answers the question: 
    #     “How many of the positive cases did I detect?” 
    #     Or to put it in a manufacturing setting: “How many (truly) defective products did I manage to recall?” 

    return round(accuracy, decimals) \
          , round(precision, decimals) \
          , round(recall_sensitivity, decimals) \
          , round(fscore, decimals) \
          , round(specificity, decimals) \
          , round(balanced_accuracy, decimals)

In [10]:
tn = confusionMatrix["TrueNegative"]
tp = confusionMatrix["TruePositive"]
fn = confusionMatrix["FalseNegative"]
fp = confusionMatrix["FalsePositive"]

make_confusion_matrix(
    tp = tp
    , tn = tn
    , fp = fp
    , fn = fn
)

           +----------------------------+
           |(TN)         |          (FP)|
         0 |    13504    |    1000      |
           |             |              |
 Actual    |-------------|--------------|
           |             |              |
         1 |     458     |     186      |
           |(FN)         |          (TP)|
           |_____________|______________|
                  0              1       
                     Predicted           


In [11]:
accuracy, precision, recall_sensitivity, fscore, specificity, balanced_accuracy = getAccuracy(
      tp = tp
    , tn = tn
    , fp = fp
    , fn = fn
    , decimals = 4
)

print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall_sensitivity}')
print(f'F1: {fscore}')
print(f'Specificity: {specificity}')
print(f'Balanced Accuracy: {balanced_accuracy}')

Accuracy: 0.9037
Precision: 0.1568
Recall: 0.2888
F1: 1.1553
Specificity: 0.9311
Balanced Accuracy: 0.6099
